In [ ]:
%%time
import mysql.connector
import pandas as pd

print("Iniciando proceso...") 

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)
cursor = conn.cursor()
query = "UPDATE secuencias SET flg_seleccionado = NULL"
cursor.execute(query)

query = "SELECT id_especie FROM maestra_especies m JOIN especies_seleccionadas s ON m.especie = s.especie"
cursor.execute(query)
ids = cursor.fetchall()
for id_especie in ids:
    print("\rPCT for " + str(id_especie[0]) + "                                              ", end=" ")
    query = "UPDATE secuencias SET flg_seleccionado = 1 WHERE id_especie = %s AND flg_pct = 1 ORDER BY RAND() LIMIT 4208"
    cursor.execute(query, [id_especie[0]])
    print("\rlncRNA for " + str(id_especie[0]) + "                                              ", end=" ")
    query = "UPDATE secuencias SET flg_seleccionado = 1 WHERE id_especie = %s AND flg_pct = 0 ORDER BY RAND() LIMIT 4208"
    cursor.execute(query, [id_especie[0]])

print("\rProceso terminado...                                                                                  \n") 
    
print("SELECCIONADOS")
df = pd.read_sql_query("SELECT m.especie, SUM(CASE WHEN s.flg_pct = 1 THEN 1 ELSE 0 END) AS PCT, SUM(CASE WHEN s.flg_pct = 0 THEN 1 ELSE 0 END) AS lncRNA FROM secuencias s JOIN maestra_especies m ON s.id_especie = m.id_especie WHERE flg_seleccionado = 1 GROUP BY m.especie ORDER BY 1 DESC", conn)
display(df)

print("NO SE USARAN EN EL MODELO")
df = pd.read_sql_query("SELECT m.especie, SUM(CASE WHEN s.flg_pct = 1 THEN 1 ELSE 0 END) AS PCT, SUM(CASE WHEN s.flg_pct = 0 THEN 1 ELSE 0 END) AS lncRNA FROM secuencias s JOIN maestra_especies m ON s.id_especie = m.id_especie WHERE flg_seleccionado IS NULL OR flg_seleccionado = 0 GROUP BY m.especie ORDER BY 1 DESC", conn)
display(df)
    
conn.close()